# 📘 代理架构 4: 规划

在这个notebook中，我们探索**规划**架构。这种模式在代理的推理过程中引入了关键的前瞻性层。与ReAct模型中逐步响应信息不同，规划代理首先将复杂任务分解为一系列更小、可管理的子目标。它在采取任何行动*之前*创建完整的'作战计划'。

这种主动方法为多步骤任务带来了结构、可预测性和效率。为了突出其优势，我们将直接比较**reactive代理(ReAct)**与我们的新**planning agent**. 我们将向两者提出一个需要在执行最终计算之前收集多条信息的任务，展示预先计算的计划如何导致更稳健和直接的解决方案。

### 定义
**规划**架构涉及一个代理，它明确地将复杂目标分解为详细的子任务 *之前*开始执行。这个初始规划阶段的输出是一个具体的、逐步的计划，代理然后有条不紊地遵循该计划以达到解决方案。

### 高级工作流程

1. **接收目标：** 代理被赋予一个复杂任务。
2. **规划：** 专门的'规划器'组件分析目标并生成有序的子任务列表以实现它。例如：`["Find fact A", "Find fact B", "Calculate C using A和B"]`.
3. **执行：** '执行器'组件接受计划并按顺序执行每个子任务，根据需要使用工具。
4. **综合：** 一旦计划中的所有步骤完成，最终组件将执行步骤的结果综合成一个连贯的最终答案。

### 何时使用/应用场景
* **多步骤工作流程：** 适用于任务操作序列已知且关键的任务，例如生成需要获取数据、处理数据然后总结的报告。
* **项目管理助手：** 将"启动新功能"等大目标分解为子任务，分配给不同的团队。
* **教育辅导：** 创建教学计划，从基础原理到高级应用教授学生特定概念。

### 优点和缺点
* **优点：**
 * **Structured & Traceable:** entire 工作流程预先布置好，使代理的过程透明且易于调试。
 * **高效：** 对于可预测的任务，可以比ReAct更高效，因为它避免了步骤之间不必要的推理循环。
* **缺点：**
 * **对变化脆弱：** 如果环境在执行期间意外变化，预制计划可能会失败。它不如ReAct代理具有适应性，后者可以在每一步后改变主意。

## 阶段0：基础与设置

我们将从标准设置过程开始：安装库并用于Nebius、LangSmith和我们的Tavily网络搜索工具配置API密钥。

### 步骤0.1： 安装核心库

**我们将要做的：**
我们将安装标准的库套件，包括更新的`langchain-tavily`包以解决弃用警告。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

### 步骤0.2： 导入库和设置密钥

**我们将要做的：**
我们将导入必要的模块并从`.env` 文件加载我们的API密钥。

**需要执行的操作：** Create 一个`.env` file在这个directory与your keys:
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [ ]:
import os
import re fromtyping import List, Annotated, TypedDict, Optional fromdotenv import load_dotenv

# LangChain components fromlangchain_nebius import ChatNebius fromlangchain_core.messages import BaseMessage, ToolMessage frompydantic import BaseModel, Field fromlangchain_core.tools import tool fromlangchain_core.messages import SystemMessage fromlangchain_tavily import TavilySearch

# LangGraph components fromlanggraph import StateGraph, END fromlang.message import AnyMessage, add_messages fromlangprebuilt import ToolNode, tools_condition

# 用于美观打印 fromrich.console import Console fromrich.markdown import Markdown

# --- API KeyandTracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Planning (Nebius)"

# 检查密钥is否已设置 forkeyin["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]:
 if not os.environ.get(key):
 print(f"{key} 未找到。请创建.env文件并设置密钥。")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1： 基线 - 一个Reactive Agent (ReAct)

要理解规划的价值，我们首先需要一个基准。我们将使用在上一个notebook中构建的ReAct代理。这个代理很聪明但目光短浅——它一次一步地找出自己的路径。

### 步骤1.1： 重建ReAct代理

**我们将要做的：**
我们将快速重建ReAct代理。它的核心特性是一个循环，在每次工具调用后，代理的输出被路由回自身，允许它重新评估并决定下一步行动，基于最新信息。

In [ ]:
console = Console()

# def我们图的state
class AgentState(TypedDict):
 messages: Annotated[list[AnyMessage], add_messages]

# 1. fromtavily包def基础工具
tavily_search_tool = TavilySearch(max_results=2)

# 2. 修复：简化自def工具。 
#。invoke()方法already returns a clean string, so we just pass it through.
@tool
def web_search(query: str) -> str:
 """使用Tavily执行网络搜索并returnresultstring。"""
 console.print(f"--- TOOL: Searchingfor'{query}'...")
 result= tavily_search_tool.invoke(query)
 returnresults

# 3. defLLM并将其绑定到我们的自def工具
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0)
llm_with_tools = llm.bind_tools([web_search])

# 4. 带有系统提示的代理节点，强制一次调用一个工具
def react_agent_node(state: AgentState):
 console.print("--- 反应式代理：思考中... ---")
 
 messages_with_system_prompt = [
 SystemMessage(content="你is一个有帮助的研究助手。你必须一次只调用一个工具。not要in一次调用中调用多个工具。in收到工具结果后，你将决定下一步。")
 ] + state["messages"]

 response= llm_with_tools.invoke(messages_with_system_prompt)
 
 return{"messages": [response]}

# 5. inToolNode中使用我们修正的自def工具
tool_node = ToolNode([web_search])

# ReAct graphwithits characteristic loop
react_graph_builder = StateGraph(AgentState)
react_graph_builder.add_node("agent", react_agent_node)
react_graph_builder.add_node("tools", tool_node)
react_graph_builder.set_entry_point("agent")
react_graph_builder.add_conditional_edges("agent", tools_condition)
react_graph_builder.add_edge("tools", "agent")

react_agent_app = react_graph_builder.compile()
print("Reactive (ReAct)代理编译成功.")

Reactive (ReAct)代理编译成功.


### 步骤1.2： 在以规划为中心的问题上测试反应式代理

**我们将要做的：**
我们将给ReAct代理一个需要两个不同的数据收集步骤，然后进行最终计算的任务。这将测试它管理多步骤工作流程的能力，而无需预先计划。

In [4]:
plan_centric_query = """
查找法国、德国and意大利首都的人口。
然后计算它们的总and。
finally，将总andwith美国人口进行比较，并说明哪个更大。
"""

console.print(f"[bold yellow]测试 REACTIVE agentina plan-centric query:[/bold yellow] '{plan_centric_query}'\n")

final_react_output = None forchunkinreact_agent_app.stream({"messages": [("user", plan_centric_query)]}, stream_mode="values"):
 final_react_output = chunk
 console.print(f"--- [bold purple]当前state更新[/bold purple] ---")
 chunk['messages'][-1].pretty_print()
 console.print("\n")

console.print("\n--- [bold red]反应式代理的最终output[/bold red] ---")
console.print(Markdown(final_react_output['messages'][-1].content))

测试 REACTIVE agent在a plan-centric query: '
查找法国、德国和意大利首都的人口。
然后计算它们的总和。
最后，将总和与美国人口进行比较，并说明哪个更大。
'

--- 当前状态更新 ---

================================ Human Message =================================


查找法国、德国and意大利首都的人口。
然后计算它们的总and。
finally，将总andwith美国人口进行比较，并说明哪个更大。



--- 反应式代理：思考中... ---

--- 当前状态更新 ---

...

--- 反应式代理：思考中... ---

--- 当前状态更新 ---

================================== Ai Message ==================================

population的Paris is approximately 2,165,423. population的Berlin is approximately 3,580,190. population的Rome is approximately 4,347,100. combined totalof the population这些three cities is 2,165,423 + 3,580,190 + 4,347,100 = 10,092,713. 美国的人口约for 331,449,281. 因此，美国的人口大于...的总人口 Paris, Berlin,andRome.


--- 反应式代理的最终输出 ---

population的Paris is approximately 2,165,423. population的Berlin is approximately 3,580,190. 
population的Rome is approximately 4,347,100. combined totalof the population这些three cities is 
2,165,423 + 3,580,190 + 4,347,100 = 10,092,713. 美国的人口约为 331,449,281. 
因此，美国的人口大于...的总人口 Paris, 
Berlin,和Rome.

**输出讨论：**
ReAct代理成功完成了任务。通过观察流式输出，我们可以追踪其逐步推理过程：
1. 它首先决定搜索巴黎的人口。
2. 在接收到该结果后，它将其纳入记忆，然后决定下一步是搜索柏林的人口。
3. 最后，收集到两条信息后，它执行计算并提供最终答案。

虽然它有效，但这种迭代发现过程并不总是最有效的。对于这样可预测的任务，代理在每一步之间进行额外的LLM调用来推理。这为展示规划代理的价值奠定了基础。

## 阶段2： 高级方法 - 一个规划 Agent

现在，让我们构建一个在行动前思考的代理。这个代理将有一个专门的**Planner**来创建完整的任务列表，一个**Executor**来执行计划，以及一个**Synthesizer**来组装最终结果。

### 步骤2.1： 定义规划器、执行器和综合器节点

**我们将要做的：**
我们将创建新代理的核心组件：
1. **`Planner`:** 一个基于LLM的节点，接受用户请求并输出结构化计划。
2. **`Executor`:** 一个节点，接受计划，使用工具执行*下一个*步骤，并记录结果。
3. **`Synthesizer`:** 一个最终的基于LLM的节点，接受所有收集的结果并生成最终答案。

In [5]:
# Pydantic模型以确保规划器的outputis结构化的步骤list
class Plan(BaseModel):
 """执行以回答用户查询的工具调用计划。"""
 steps: List[str] = Field(description="执行后将回答查询的工具调用list。")

# def规划代理的state
class PlanningState(TypedDict):
 user_request: str
 plan: Optional[List[str]]
 intermediate_steps: List[ToolMessage]
 final_answer: Optional[str]

def planner_node(state: PlanningState):
 """生成行动计划以回答用户的请求。"""
 console.print("--- 规划器：分解任务中... ---")
 planner_llm = llm.with_structured_output(Plan)
 
 # THE FIX: A much more explicit promptwitha clear示例(few-shot prompting)
 prompt = f"""你is一名专业的规划师。你的工作is创建逐步计划来回答用户的请求。
 计划中的每一步都必须is对`web_search`工具的单次调用。

 **说明：**
 1. 分析用户的请求。
 2. 将其分解for一系列简单、合乎逻辑的搜索查询。
 3. 将output格式化forstringlist，其中每个string都is单个有效的工具调用。

 **示例：**
 请求："法国的首都is什么，它的人口is多少？"
 正确的计划output：
 [
 "web_search('capital的France')",
 "web_search('population的Paris')"
 ]

 **用户的请求：**
 {state['user_request']}
 """

 plan_result = planner_llm.invoke(prompt)
 # Use plan_result.steps, not plan.steps来avoid confusionwiththe variable name 'plan'
 console.print(f"--- 规划器：生成的计划： {plan_result.steps} ---")
 return{"plan": plan_result.steps}

def executor_node(state: PlanningState):
 """执行计划中的下一步。"""
 console.print("--- 执行器：运行下一步... ---")
 计划= state["plan"]
 next_step = plan[0]
 
 # Robust regex来handle both singleanddouble quotes
 match = re.search(r"(\w+)\((?:\"|\')(.*?)(?:\"|\')\)", next_step)
 if not match:
 tool_name = "web_search"
 查询= next_step
 else:
 tool_name,查询= match.groups()[0], match.groups()[1]
 
 console.print(f"--- 执行器：调用工具 '{tool_name}'withquery '{query}' ---")
 
 result= tavily_search_tool.invoke(query)
 
 # We still创建a ToolMessage，但the工具call itself is now safe.
 tool_message = ToolMessage(
 content=str(result),
 name=tool_name,
 tool_call_id=f"manual-{hash(query)}"
 )
 
 return{
 "plan": plan[1:], # Pop the executed stepfromthe plan
 "intermediate_steps": state["intermediate_steps"] + [tool_message]
 }

def synthesizer_node(state: PlanningState):
 """from中间步骤综合最终答案。"""
 console.print("--- 综合器：生成最终答案中... ---")
 
 context = "\n".join([f"Tool {msg.name} returned: {msg.content}"formsginstate["intermediate_steps"]])
 
 prompt = f"""你is一名专业的综合器。基于用户的请求and收集的data，提供全面的最终答案。
 
 请求：{state['user_request']}
 收集的data：
 {context}
 """
 final_answer = llm.invoke(prompt).content
 return{"final_answer": final_answer}

print("规划器、执行器and综合器节点已def。")

规划器、执行器and综合器节点已def。


### 步骤2.2： 构建规划代理图

**我们将要做的：**
现在我们将把新节点组装成一个图。流程将是： `Planner` -> `Executor`（循环）-> `Synthesizer`.

In [6]:
def planning_router(state: PlanningState):
 if not state["plan"]:
 console.print("--- 路由器：计划完成。移至综合器。 ---")
 return "synthesize"
 else:
 console.print("--- 路由器：计划还有更多步骤。continue执行。 ---")
 return "execute"

planning_graph_builder = StateGraph(PlanningState)
planning_graph_builder.add_node("plan", planner_node)
planning_graph_builder.add_node("execute", executor_node)
planning_graph_builder.add_node("synthesize", synthesizer_node)

planning_graph_builder.set_entry_point("plan")
planning_graph_builder.add_conditional_edges("plan", planning_router, {"execute": "execute", "synthesize": "synthesize"}) # Route in...之后planning
planning_graph_builder.add_conditional_edges("execute", planning_router, {"execute": "execute", "synthesize": "synthesize"})
planning_graph_builder.add_edge("synthesize", END)

planning_agent_app = planning_graph_builder.compile()
print("Planning代理编译成功.")

Planning代理编译成功.


## 阶段3： 正面比较

让我们在相同任务上运行我们的新规划代理，并将其执行流程和最终输出与反应式代理进行比较。

In [7]:
console.print(f"[bold green]测试 PLANNING agentinthe same plan-centric query:[/bold green] '{plan_centric_query}'\n")

# Remember来initialize thestatecorrectly, especially the listforintermediate steps
initial_planning_input = {"user_request": plan_centric_query, "intermediate_steps": []}

final_planning_output = planning_agent_app.invoke(initial_planning_input)

console.print("\n--- [bold green]规划代理的最终output[/bold green] ---")
console.print(Markdown(final_planning_output['final_answer']))

测试 PLANNING agent在the same plan-centric query: '
查找法国、德国和意大利首都的人口。
然后计算它们的总和。
最后，将总和与美国人口进行比较，并说明哪个更大。
'

--- 规划器：分解任务中... ---

--- 规划器：生成的计划： ["web_search('populationof the capital的France')", "web_search('populationof the 
capital的Germany')", "web_search('populationof the capital的Italy')", "web_search('populationof the United 
States')", "web_search('add + population的France + population的Germany + population的Italy')", 
"web_search('compare + combined total + 美国的人口')"] ---

--- 路由器：计划还有更多步骤。继续执行。 ---

--- 执行器：运行下一步... ---

...

--- 执行器：调用工具 'web_search'与query 'compare + combined total + 美国的人口' ---

--- 路由器：计划完成。移至综合器。 ---

--- SYNTHESIZER: 生成 final answer... ---

--- 规划代理的最终输出 ---

基于收集的数据，以下是对用户请求的最终答案： 

 1 法国、德国和意大利首都的人口是： 
  • 法国巴黎： 2,048,472 (根据Instagram) 
  • 德国柏林： 3,800,000 (根据Google Arts & Culture) 
  • 意大利罗马： 2,800,000 (根据Instagram) 
 2 这三个首都城市的总人口是： 
  • 2,048,472 (Paris) + 3,800,000 (Berlin) + 2,800,000 (Rome) = 8,648,472 
 3 美国的人口约为 343,603,404 (根据Macrotrends). 
 4 三个首都城市的总人口 (8,648,472) 明显小于 
 美国的人口 (343,603,404). 

因此，美国的人口大于...的总人口 法国、德国和意大利的首都城市。

**输出讨论：**
过程的差异立即显现。第一步就是`规划器`创建完整、明确的计划：`['web_search("population的Paris")', 'web_search("population的Berlin")']`. 

代理然后有条不紊地执行这个计划，无需在步骤之间停下来思考。这个过程是：
- **更透明：** 我们可以在代理开始之前看到其整个策略。
- **更稳健：** 它不太可能偏离轨道，因为它遵循一套明确的指令。
- **可能更高效：** 它避免了步骤之间推理的额外LLM调用。

这展示了规划对于可以预先确定所需步骤的任务的强大功能。

## 阶段4： 定量评估

为了正式化我们的比较，我们将使用LLM作为评判者来评分两个代理，重点关注其问题解决过程的质量和效率。

In [8]:
class ProcessEvaluation(BaseModel):
 """评估代理问题解决过程的模式。"""
 task_completion_评分: int = Field(description="对代理is否成功完成任务进行1-10评分。")
 process_efficiency_评分: int = Field(description="对代理过程的效率and直接性进行1-10评分。更高的评分意味着更合乎逻辑且更少迂回的路径。")
 理由: str = Field(description="评分的简要理由。")

judge_llm = llm.with_structured_output(ProcessEvaluation)

def evaluate_agent_process(query: str, final_state: dict):
 # forthe ReAct agent, the trace isin'messages'. forPlanning, it'sin'intermediate_steps'.
 if 'messages'infinal_state:
 trace = "\n".join([f"{m.type}: {str(m.content)}"forminfinal_state['messages']])
 else:
 trace = f"Plan: {final_state。get('plan', [])}\nSteps: {final_state。get('intermediate_steps', [])}"
 
 prompt = f"""你is一名专业的AI代理评判员。in1-10的等级上评估代理解决任务的过程。
 重点关注过程is否合乎逻辑and高效。
 
 **用户的任务：** {query}
 **完整代理跟踪：**\n```\n{trace}\n```
 """
 returnjudge_llm.invoke(prompt)

console.print("--- 评估 Reactive Agent's Process ---")
react_agent_evaluation = evaluate_agent_process(plan_centric_query, final_react_output)
console.print(react_agent_evaluation.model_dump())

console.print("\n--- 评估 Planning Agent's Process ---")
planning_agent_evaluation = evaluate_agent_process(plan_centric_query, final_planning_output)
console.print(planning_agent_evaluation.model_dump())

--- 评估 Reactive Agent's Process ---

{
 '任务完成评分': 10,
 '过程效率评分': 8,
 '理由': '代理成功完成了任务，通过找到首都城市的人口 
法国、德国和意大利，然后计算它们的总和。然而，过程并不完全 
高效，因为代理进行了多次工具调用来查找每个城市的人口，而答案没有被有效 
使用来计算总和。代理本可以使用第一次工具调用的信息来 
calculate the combined total和then compared itto the 美国的人口. Additionally, the代理
本可以使用更高效的工具或方法来查找城市的人口，例如使用单个工具 
一次提供多个城市的人口。'
}

--- 评估 Planning Agent's Process ---

{
 '任务完成评分': 8,
 '过程效率评分': 6,
 '理由': '代理能够找到法国、德国和 
意大利首都城市的人口，并计算它们的总和。然而，过程并不完全高效，因为代理必须 
执行多次网络搜索以找到正确的信息。代理也没有直接比较总和
与美国人口，而是必须执行另一次网络搜索以找到相关 
信息。总的来说，代理能够完成任务，但过程本可以更高效。'
}

**输出讨论：**
评判员的评分量化了两种方法的差异。两个代理可能都会获得高`task_completion_评分`，因为它们最终都找到了答案。然而，**规划 Agent**将获得显著更高的`process_efficiency_评分`。评判员的理由将强调其预先计划是解决问题的更直接和合乎逻辑的方式，相比ReAct代理的逐步探索过程。

这个评估证实了我们的假设：对于解决路径可预测的问题，规划架构提供了更结构化、透明和高效的方法。

## 总结

在这个notebook中，我们已经实现了**规划**架构，并将其与**ReAct**模式直接对比。通过强制代理在执行前首先构建全面的计划，我们在透明度、稳健性和效率方面获得了显著的好处，特别是对于定义明确的多步骤任务.

虽然ReAct在下一步未知的探索性场景中表现出色，但当解决方案的路径可以预先规划时，规划架构就会大放异彩。理解这种权衡对系统设计者至关重要。为正确的问题选择正确的架构是构建有效和智能AI代理的关键技能。规划模式是该工具包中的重要工具，为复杂、可预测的工作流程提供所需的结构。